# Introduction

This script is used to gather data from the stack, in essence the Air Quality data.

# Initialization

## Import libraries

In [3]:
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import json
from datetime import datetime
from datetime import timedelta  
import time
import sys
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

pd.options.mode.chained_assignment = None 

## Set up parameters

In [4]:
##### Choose scope #####
scopeID = 'cot.belair'

In [12]:
##### Insert token and generate URL #####
myToken = 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICIwUnhaVzd0N1d5aEczaEo3cUhoM3hQa3MzbkthTUE5Zy04SnozY2trQ3EwIn0.eyJqdGkiOiIxNzY0MTRmZC00MmEyLTQzYjktODkxZi1mNWU5NTU2ZjM4MDEiLCJleHAiOjE1NTI1NTM1OTUsIm5iZiI6MCwiaWF0IjoxNTUyNTUyOTk1LCJpc3MiOiJodHRwczovL2lkbGFiLWlvdC50ZW5ndS5pby9hdXRoL3JlYWxtcy9pZGxhYi1pb3QiLCJhdWQiOiJwb2xpY3ktZW5mb3JjZXIiLCJzdWIiOiI3NGVjNTQzYi03Yjc1LTQ1ZGItOWExNy0xMDY5OTlmYmU3OWEiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJzd2FnZ2VyLXVpIiwibm9uY2UiOiJjNDhhNmEzZi00ODdlLTQ1NzktOTQ1ZS1mYjg5Zjc0MzBkYWQiLCJhdXRoX3RpbWUiOjE1NTI1NTE4NzEsInNlc3Npb25fc3RhdGUiOiI2MWYyYTRlZC1mNWQwLTQ5YzgtOWE2Ni1hZGYyY2RiZjk3YjkiLCJhY3IiOiIwIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vaWRsYWItaW90LnRlbmd1LmlvIiwiaHR0cDovL2xvY2FsaG9zdDo1NTU1Il0sInJlc291cmNlX2FjY2VzcyI6eyJwb2xpY3ktZW5mb3JjZXIiOnsicm9sZXMiOlsidXNlcjp2aWV3Il19fSwiYXV0aG9yaXphdGlvbiI6eyJwZXJtaXNzaW9ucyI6W3sicnNpZCI6IjQxOGFiMGYzLWE5NTQtNGEwMS04ZTdlLWFlZGQzN2VjZTcyMyIsInJzbmFtZSI6ImRhdGE6c2NvcGVkOnZpZXcifV19LCJzY29wZSI6Im9wZW5pZCBwcm9maWxlIGVtYWlsIGNvdC1zY29wZSIsImVtYWlsX3ZlcmlmaWVkIjpmYWxzZSwicmlnaHRzIjpbXSwibmFtZSI6IkplZmYgR2V1ZGVucyIsImdyb3VwcyI6WyIvYWxsLXVzZXJzIiwiL2NvdC90aGVzaXMiXSwicHJlZmVycmVkX3VzZXJuYW1lIjoiZ2V1ZGVucy5qZWZmQGdtYWlsLmNvbSIsImdpdmVuX25hbWUiOiJKZWZmIiwiZmFtaWx5X25hbWUiOiJHZXVkZW5zIiwiZW1haWwiOiJnZXVkZW5zLmplZmZAZ21haWwuY29tIiwicGljdHVyZSI6Imh0dHBzOi8vbGg1Lmdvb2dsZXVzZXJjb250ZW50LmNvbS8tRTYxVzBvTmFlOUkvQUFBQUFBQUFBQUkvQUFBQUFBQUFCd2MvUExDNy1mNTdnek0vcGhvdG8uanBnP3N6PTUwIn0.VaIMDTXSSWKED2g1CJJCLQ5urRxU938AbmaRnC2jYPdNh62i9iLW78HgxNrpuPvzWPRz71p9jl60eAvD21zcarFGVfaUJB-iSstjGTixEXiCAlefsh0fn3E6QwV8y-n3XvdRKULxkqzhyl7R7lARcBabFRvLqFMY389_sbkRcFQX8OiKyh_AsWD4YrGR0ZLuzyso9r13hnBfuV4QflX27MukoR1FgHOiKESecNl0veDj7NTNKHA2wZC-pi4ufmA9uTUpRLXVvTpynSYP0CinoexLBcHBhSAfjS3TFCN7oARfk4kuiI2QpeCj9TK_F6YpdCQLbbZa0Ahb8265W1r-sA'

In [6]:
##### Specify time range #####
##### Adjust this time range to specify which period you want to query - Mostly I use periods of one month #####
startDate = datetime(2018, 12, 17, 1, 0)
endDate = datetime(2019, 2, 1, 1, 0)
startTime = int(startDate.timestamp())*1000;  #print(startTime)
endTime = int(endDate.timestamp())*1000;  #print(endTime)

# Lora Air quality sensor

In [7]:
thingID = 'lora.343233386B376717'

### Choose metrics

In [8]:
#metricsList = ['environment.light','environment.temperature','weather.uv','environment.relativehumidity']
metricsList = ['airquality.no2', 'airquality.o3', 'airquality.pm1', 'airquality.pm10', 
               'airquality.pm25', 'environment.relativehumidity', 'environment.temperature']

### Run query

In [13]:
for i in range(0,len(metricsList)):
    print(metricsList[i]+'_Belair')
    metricID = metricsList[i]

    myUrlQuery = 'https://idlab-iot.tengu.io/api/v1/scopes/{scope}/query/{metric}/events?from={start}&to={end}&things={thing}&orderByTime=asc'\
        .format(scope = scopeID, metric = metricID, start=startTime, end=endTime, thing=thingID)

    headers = {
        'accept': 'application/json',
        'authorization': 'Bearer {0}'.format(myToken),
    }

    response = requests.get(myUrlQuery, headers=headers)
    #print(response.content)
    data = response.json() 
    column_names = data['columns']; #print(column_names)
    values = data['values']
    df = pd.DataFrame.from_dict(values)
    df.columns = column_names
    air_data = df[['time','value']]
#     print(weather_data)
    path=r'C:\Users\JeffG\Desktop\Case 2 - data\data'
    air_data.to_csv(path+'\\'+str(thingID) +'_' + metricsList[i]+'_Belair'+'.csv')

airquality.no2_Belair
airquality.o3_Belair
airquality.pm1_Belair
airquality.pm10_Belair
airquality.pm25_Belair
environment.relativehumidity_Belair
environment.temperature_Belair
